# import

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import math
import copy
import matplotlib.pyplot as plt
import tdc

from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.utils.data.dataset import Subset
from sklearn.model_selection import KFold
from torch.nn.utils.rnn import pad_sequence

from torch.utils.tensorboard import SummaryWriter

import tdc.single_pred
import tdc.benchmark_group

print(torch.__version__)
print(torch.cuda.is_available())
torch.cuda.device_count()

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
import rdkit.Chem.Fragments

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from sklearn.manifold import TSNE
import sklearn

2.2.1
True


# prepare
* functions for computing precision recall accuracy
* read files of Pretrain_SMILES_STR_LIST and SymbolDictionary which are obtained during pretraining
* Utility

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Threshold = torch.tensor([0.5]).to(device) 

def positive(proba):
    a = (proba >= Threshold) 
    return a

def negative(proba):
    a = (proba < Threshold) 
    return a

def seek_TP(pred_y, y):
  tp = (positive(pred_y) & positive(y)).int()
  return tp

def seek_TN(pred_y, y):
  tn = (negative(pred_y) & negative(y)).int()
  return tn



LIMIT_SMILES_LENGTH = 100

import pickle
f = open('CHEMBL/OdorCode-40 Pretrain_SMILES_STR_LIST','rb')
Pretrain_SMILES_STR_LIST = pickle.load(f)
f.close()

f = open('CHEMBL/OdorCode-40 Symbol Dictionary', 'rb')
[symbol_ID, ID_symbol, sID] = pickle.load(f)
f.close()

PAD_ID = 0
CLS_ID = 1
BOS_ID = 2
EOS_ID = 3
MSK_ID = 4



#----------------------------------#
#           smiles_str2smiles      #
#----------------------------------#
# translate a string of SMILES to a list of symbol ID

max_length_symbol = max([len(s) for s in ID_symbol])

def smiles_str2smiles(smiles_str, flag=False):
  "translate a string of SMILES to a list of symbol ID（symbols such as 'Na' will translated to ID of 'Na')

  smiles = []
  i=0
  while i < len(smiles_str):
    NotFindID = True
    for j in range(max_length_symbol,0,-1) :
      if i+j <= len(smiles_str) and smiles_str[i:i+j] in symbol_ID: # 
        smiles.append(symbol_ID[smiles_str[i:i+j]])
        i += j-1 
        NotFindID = False
        break
    if NotFindID:
      smiles.append(4) # using MSK_ID to replace unknown symbol
    i += 1
  return smiles

#----------------------------------#
#           smiles2smiles_str      #
#----------------------------------#
def smiles2smiles_str(smiles):
  smiles_str = ''
  for id in smiles:
    smiles_str += ID_symbol[id]
  return smiles_str


In [3]:
def logResult(filepath, filename, which_model, data_name, result_dict):
    if os.path.exists(filepath):
        pass
    else:
        os.makedirs(filepath)

    temp = os.path.isfile(filepath+filename)
    
    f = open(filepath+filename, 'a')
    if temp == False:
        title_list = [
            'task_name', 
            'task_type',
            'pretrain', 
            'macroF1 or RMSE',
            'microF1 or r^2',
            'ROCAUC_macro',
            'ROCAUC_micro',
            'PRAUC macro',
            'PRAUC micro',
            'tdc',
            '\n'
        ]
        f.write(', '.join(title_list))

    if data_name in regression_list:
        task_type = 'regression'
    else:
        task_type = 'classification'

    content_list = [
        data_name,
        task_type,
        which_model,
        '%.4f' % result_dict['macroF1 or RMSE'],
        '%.4f' % result_dict['microF1 or r^2'],
        '%.4f' % result_dict['ROCAUC_macro'],
        '%.4f' % result_dict['ROCAUC_micro'],
        '%.4f' % result_dict['PRAUC macro'],
        '%.4f' % result_dict['PRAUC micro'],
        '%s' % result_dict['tdc'],
        '\n'
    ]
    f.write(', '.join(content_list))

    f.close()

# dataMake
* TDC: prepare for predicting molecular properties of TDC ADMET Benchmark Group
* ODs: prepare for predicting odor descriptors

## TDC

### tdc.benchmark_group

In [4]:
regression_list = [
    'caco2_wang',
    'lipophilicity_astrazeneca',
    'solubility_aqsoldb',
    'ppbr_az',
    'vdss_lombardo',
    'half_life_obach',
    'clearance_hepatocyte_az',
    'clearance_microsome_az',
    'ld50_zhu',

]

classification_list = [
    'hia_hou',
    'pgp_broccatelli',
    'bioavailability_ma',
    'bbb_martins',
    'cyp2d6_veith',
    'cyp3a4_veith',
    'cyp2c9_veith',
    'cyp2c9_substrate_carbonmangels',
    'cyp2d6_substrate_carbonmangels',
    'cyp3a4_substrate_carbonmangels',
    'herg',
    'ames', 
    'dili', 

]

data_name_list = regression_list + classification_list

Found local copy...


In [5]:
def dataMake_group(d_name, strSmiles_list, labels):
    if d_name in regression_list:
        task_type = 'regression'
    else:
        task_type = 'classification'

    label_ID = {d_name: 0}
    lID = 1
    ID2label = [d_name]

    smiles_list = []
    labels_vec_list = []
    canonical_smiles_list = []
    for i in range(len(strSmiles_list)):
        smi = strSmiles_list[i]
        mol = Chem.MolFromSmiles(smi)
        if mol == None:
            continue
        else:
            smi = Chem.MolToSmiles(mol)
            if len(smi) > LIMIT_SMILES_LENGTH:
                smi = smi[:LIMIT_SMILES_LENGTH]
                # continue
        
        temp = smiles_str2smiles(smi)
        if temp is not None:
            canonical_smiles_list.append(smi)
            smiles_list.append(temp)

            labels_vec_list.append([])
            labels_vec_list[-1].append(float(labels[i]))
        
    return(
        smiles_list, 
        canonical_smiles_list,
        labels_vec_list,
        lID,
        label_ID,
        task_type,
        ID2label
        )

## ODs

In [ ]:
F_odor     = 'webScrapping/tgsc_odorant_1020.txt'    # path of ODs data (refer to README.md about obtaining ODs data)       
F_odorless = 'webScrapping/tgsc_odorless_1020.txt'  

LIMIT_FREQ = 49   # predict ODs whose number of positive samples over LIMIT_FREQ 

#-----------------------------------------------------------------------------------------
# compute frequence of ODs
#----------------------------------------------------------------------------------------- 
label_ID ={}    
freq_label = {} 

with open(F_odor,'r') as inF:
  while True:
    line = inF.readline()
    if line == '':
      break
    # x = line.split("\t")
    x = line.split()
    smiles_str = x[1]

    # the length of smiles_str longer than LMIT_SMILES_LENGTH will be omitted
    if len(smiles_str) > LIMIT_SMILES_LENGTH :
      continue
    else:
      mol = Chem.MolFromSmiles(smiles_str)
      if mol == None:
        continue
      else:
        # odor_descriptors = x[2].split()
        odor_descriptors = x[2:]
        for odd in odor_descriptors:
          if odd == 'odorless':
            continue
          if odd in freq_label:
            freq_label[odd] += 1
          else:
            freq_label[odd] = 1

#----------------------------------------------
# create ID for ODs
#----------------------------------------------
ID2label = []
lID = 0

freq_label_sorted = sorted(freq_label.items(), key=lambda x:x[1], reverse=True)
for label, freq in freq_label_sorted:
  if freq > LIMIT_FREQ:
    label_ID[label] = lID
    ID2label.append(label)
    lID += 1

print("number of odor descriptors: ", lID)
for label, freq in freq_label_sorted:
  if freq>49:
     print('%5d : '%freq, label)


In [ ]:
#------------------------------------------------------------------------------------------------------
#   make target (labels_vec_list)
#-----------------------------------------------------------------------------------------------------


def labels2vec(label_sequence):
  # label_list = label_sequence.split()
  label_list = label_sequence.copy()
  labels_vec = [0.0]*lID
  for label in label_list:
    if label == 'odorless':  
      continue
    if label not in label_ID:
      continue
    labels_vec[label_ID[label]] = 1.0  
  return labels_vec

smiles_list = []
labels_vec_list = []
canonical_smiles_list = []

def make_data(filename):
  "make smiles_list and labels_vec_list"

  with open(filename,'r') as inF:
    while True:
      line = inF.readline()
      if line == '':
        break

      x = line.split()
      smiles_str = x[1]

      # the length of smiles_str longer than LMIT_SMILES_LENGTH will be omitted
      if len(smiles_str) > LIMIT_SMILES_LENGTH :
        continue
      else:
        mol = Chem.MolFromSmiles(smiles_str)
        if mol == None:
          continue
        else:
          # smiles_str to canonical smiles_str
          canonical_smiles_str = Chem.MolToSmiles(mol)
          smiles_list.append(smiles_str2smiles(canonical_smiles_str))      
          labels_vec_list.append(labels2vec(x[2:])) 
          canonical_smiles_list.append(canonical_smiles_str)

make_data(F_odor)
make_data(F_odorless)

print(smiles_list[0])
print(labels_vec_list[0])
print(smiles_list[1])
print(labels_vec_list[1])
task_type = 'classification'

# Model

In [6]:
InitRange = 0.1
NumToken = sID
NumHead = 8
Activation = 'gelu'
NormFirst = True
DimOdorCode = 100

#--------------------------------------------------------------------------------
class PositionalEncoder(nn.Module):

    def __init__(self, d_model, max_len=2048):  # d_model: dimension of embedding
        super().__init__()

        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) #.transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]

#----------------------------------------------------------------------------
class SymbolEncoder(nn.Module):
    def __init__(self, num_token, d_model):  
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(num_token, d_model, padding_idx=PAD_ID)
        self.embed.weight.data.uniform_(-InitRange, InitRange)  # embedding init

    def forward(self, src):
        src = self.embed(src) * math.sqrt(self.d_model)
        return src
#----------------------------------------------------------------------------
class MyTransformerEncoder(nn.Module):
    def __init__(self, d_model, num_head, d_hidden):
        super().__init__()

        encoder_layers = nn.TransformerEncoderLayer(d_model, num_head, dim_feedforward=d_hidden, norm_first = NormFirst, activation=Activation, dropout=Dropout, batch_first=True)
        encoder_norm = nn.LayerNorm(d_model)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, NumLayers, norm=encoder_norm)     

    def forward(self, x, padding_mask):
        x = self.transformer_encoder(x, src_key_padding_mask = padding_mask)
        return x
#--------------------------------------------------------------------------------


class Model(nn.Module):
    def __init__(self, lID, task_type):
        super().__init__()
        self.task_type = task_type

        self.positional_encoder = PositionalEncoder(DimEmbed)
        self.symbol_encoder = SymbolEncoder(NumToken, DimEmbed) 
        self.smiles_encoder = MyTransformerEncoder(DimEmbed, NumHead, DimTfHidden)
        self.drop0 = nn.Dropout(p=Dropout)

        self.drop = nn.Dropout(p=Dropout)  
        self.fnn = nn.Linear(DimEmbed, lID)
        self.act = nn.Sigmoid()

        self.drop2 = nn.Dropout(p=Dropout)
        self.fnn2 = nn.Linear(DimEmbed, DimEmbed)
        self.act2 = nn.ReLU()


    def forward(self, smiles):
        # add cls
        cls = torch.ones(smiles.size(0),1,dtype=torch.long).to(device)
        x = torch.concat((cls, smiles), dim=1) 
        padding_mask = (x == PAD_ID)
        x = self.drop0(self.symbol_encoder(x) + self.positional_encoder(x)) 
        x = self.smiles_encoder(x, padding_mask)  # x : OdorCode

        x = self.act2(self.fnn2(self.drop2(x[:,0,:])))

        if self.task_type == 'classification':
                return self.act(self.fnn(self.drop(x)))
        else:
            return self.fnn(self.drop(x))

# mainProgram

## tdcGroup

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import r2_score

def main_program(rate1, hpara, fold_cv=5, data_name='caco2_wang'):
    '''
    the learning rate for layers training in transfer learning is: LearningRate * rate1
    hpara = {
        'pretrain_path': 'modelsave0203',
        'DimEmbed': 256,
        'DimTfHidden': 512,
        'NumHead': 8,
        'NumLayers': 10,
        'MaskRate': 0.5,
        'LearningRate': 0.0001, 
        'Dropout': 0.1
    } 
    
    fold_cv is the number of seeds for train_val split
    '''

    torch.cuda.empty_cache()

    # load data
    tdc_group = tdc.benchmark_group.admet_group(path = '/tf/haha/all_data/TDC/admet_group/')
    benchmark = tdc_group.get(data_name)
    train_val, test_set = benchmark['train_val'], benchmark['test']
    # test set
    smiles_list_test, canonical_smiles_list_test,labels_vec_list_test,lID,label_ID,task_type,ID2label = dataMake_group(data_name, test_set.Drug.to_list(), test_set.Y.to_list())
    print(len(smiles_list_test))

    para_name = '%d_%d_%d_%d_%.2f_%.2f_%.2f' % (hpara['DimEmbed'], hpara['DimTfHidden'], 
    hpara['NumHead'], hpara['NumLayers'], hpara['MaskRate'], hpara['LearningRate'], hpara['Dropout'])

    list_train_loss = []
    list_train_metrics = [] # micro f1 or rmse
    list_test_loss = []
    list_test_metrics = [] # micro f1 or rmse

    if task_type == 'classfication':
        criterion_label = nn.BCELoss()
    else:
        criterion_label = nn.MSELoss()

    
    fold_cv = fold_cv

    # record result for tensorboard
    # record F1 for classification and RMSE for regression
    avgOD_metrics = torch.zeros(fold_cv, NumEpoch).to(device) # microF1 or rmse
    eachODFold = torch.zeros(fold_cv, NumEpoch, lID).to(device) #macroF1 can be obtained by averaging lID axis

    avgOD_metrics_test = torch.zeros(fold_cv, NumEpoch).to(device) # microF1 or rmse
    eachODFold_test = torch.zeros(fold_cv, NumEpoch, lID).to(device) # macroF1 can be obtained by averaging lID axis

    best_metrics = 0

    # used for changing canonical SMILES to non-canonical SMILES
    def change_smiles(smi):
        if smi.find('.') >= 0:
            return smiles_str2smiles(smi)

        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            return smiles_str2smiles(smi)
        num_atoms = mol.GetNumAtoms()
        pos_list = list(range(num_atoms))

        pos = random.choice(pos_list)
        new_smi = Chem.MolToSmiles(mol, rootedAtAtom=pos)
        if len(new_smi) < LIMIT_SMILES_LENGTH:
            return smiles_str2smiles(new_smi)
        else:
            return smiles_str2smiles(smi)

    rocauc_macro = 0
    pr_auc_macro = 0
    rocauc_micro = 0
    pr_auc_micro = 0
    r_2 = 0

    
    tdc_predictions_list = []

    for _fold in range(fold_cv):
        print('seed = ', _fold)

        # load data
        train_set, valid_set = tdc_group.get_train_valid_split(benchmark =data_name, split_type = 'default', seed =_fold)
        labels_train = train_set.Y.to_list()
        smilesStr_list = train_set.Drug.to_list()
        smiles_list, canonical_smiles_list,labels_vec_list,lID,label_ID,task_type,ID2label = dataMake_group(data_name, smilesStr_list, labels_train)

        #============================= train_dataloader ===========================
        train_dataloader = DataLoader(list(range(len(smiles_list))), batch_size, shuffle=True)

        #============================== test_dataloader  =====================================
        test_dataloader = DataLoader(list(range(len(smiles_list_test))), batch_size, shuffle=False)  

         #====================== model, optimizer  ==============
        label_estimator = Model(lID, task_type).to(device)

        # load pretrained encoder

        if hpara['pretrain_path'] == 'nopre':
            optimizer_label_estimator = optim.Adam(label_estimator.parameters(), hpara['LearningRate'])
        else:
            pf_symbol_en = hpara['pretrain_path']+'/'+'OdorCode-40 symbol_encoder D'+str(hpara['DimEmbed'])+'.Hidden'+str(hpara['DimTfHidden'])+'.Head'+ str(hpara['NumHead'])+'.L'+str(hpara['NumLayers'])+'.R'+str(hpara['MaskRate'])+'.S'+str(TotalSize)+'-epoch.'+str(ModelEpoch)
            pf_smiles_en = hpara['pretrain_path']+'/'+'OdorCode-40 smiles_encoder D'+str(hpara['DimEmbed'])+'.Hidden'+str(hpara['DimTfHidden'])+'.Head'+ str(hpara['NumHead'])+'.L'+str(hpara['NumLayers'])+'.R'+str(hpara['MaskRate'])+'.S'+str(TotalSize)+'-epoch.'+str(ModelEpoch)
            print('reading '+pf_symbol_en)
            label_estimator.symbol_encoder.load_state_dict(torch.load(pf_symbol_en))
            print('reading '+pf_smiles_en)
            label_estimator.smiles_encoder.load_state_dict(torch.load(pf_smiles_en))

            for param in label_estimator.symbol_encoder.parameters():
                param.requires_grad = False

            for param in label_estimator.smiles_encoder.parameters():
                param.requires_grad = False
            for param in label_estimator.smiles_encoder.transformer_encoder.layers[-1].parameters():
                param.requires_grad = True
            
            optimizer_label_estimator = optim.Adam([ {'params': label_estimator.smiles_encoder.transformer_encoder.layers[-1].parameters(), 'lr': hpara['LearningRate']*rate1},
                                                        {'params': label_estimator.fnn2.parameters(), 'lr': hpara['LearningRate']},
                                                        {'params': label_estimator.fnn.parameters(), 'lr': hpara['LearningRate']}])
            
        if task_type == 'classification':
            best_metrics_fold = 0 # macro f1
        else:
            best_metrics_fold = 1e9 # rmse
        best_logits_pred_fold = []
        best_epo_fold = None

        
        for epoch in range(NumEpoch):
            #---------------- train step -----------------
            sample_num = 0
            total_loss = 0

            if task_type == 'classification': 
                total_tps = torch.zeros(lID).to(device)
                total_real_pos = torch.zeros(lID).to(device)
                total_pred_pos = torch.zeros(lID).to(device)

            label_estimator.train()

            for idxs in train_dataloader:
                smiles = pad_sequence([torch.tensor([BOS_ID]+change_smiles(canonical_smiles_list[idx])+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                # smiles = pad_sequence([torch.tensor([BOS_ID]+smiles_list[idx]+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                labels = torch.tensor([labels_vec_list[idx] for idx in idxs],dtype=torch.float).to(device)

                optimizer_label_estimator.zero_grad()

                estimated_labels = label_estimator(smiles)

                loss = criterion_label(estimated_labels, labels)
                
                loss.backward()     

                optimizer_label_estimator.step()  

                total_loss += loss.item()*len(idxs)
                sample_num += len(idxs)

                if task_type == 'classification': 
                    total_tps += seek_TP(estimated_labels, labels).sum(0)
                    total_real_pos += (positive(labels).int()).sum(0)
                    total_pred_pos += (positive(estimated_labels).int()).sum(0)

            mean_loss = total_loss / sample_num
            list_train_loss.append(mean_loss)

            # compute metrics
            if task_type == 'classification':
                precision = total_tps/(total_pred_pos + 1e-9)
                recall    = total_tps/(total_real_pos + 1e-9)
                f1 = 2.0*precision*recall/(precision+recall+1e-9) # (lID, )
                eachODFold[_fold, epoch] = f1
                micro_mean_precision = total_tps.sum()/total_pred_pos.sum()
                micro_mean_recall    = total_tps.sum()/total_real_pos.sum()      
                metrics = 2*micro_mean_precision*micro_mean_recall/(micro_mean_precision+micro_mean_recall)
                metrics = metrics.item()
            else:
                metrics = math.sqrt(mean_loss)
                eachODFold[_fold, epoch, 0] = metrics 

            list_train_metrics.append(metrics)

            avgOD_metrics[_fold, epoch] = metrics # record for tensorboard
            
            # plot on tensorboard
            writter.add_scalars('fold%s-microF1orRMSE/train' % _fold, {para_name: metrics}, epoch)
            if _fold == fold_c-1:
                macro_f1 = torch.sum(eachODFold[:, epoch, :]).item() / (fold_cv*lID)
                writter.add_scalars('AvgFold-macroF1orRMSE/train', {para_name: macro_f1}, epoch) 
                for i in range(lID):
                    od_f1 = torch.sum(eachODFold[:, epoch, i]).item() / fold_cv
                    writter.add_scalars('AvgFold-%s/train' % ID2label[i], {para_name:od_f1}, epoch)


            #--------------- test step ------------------
            sample_num = 0
            total_loss = 0

            if task_type == 'classification':
                total_tps = torch.zeros(lID).to(device)
                total_real_pos = torch.zeros(lID).to(device)
                total_pred_pos = torch.zeros(lID).to(device)

            label_estimator.eval()

            labels_test = torch.zeros([1, lID]).to(device)
            preds_test = torch.zeros([1, lID]).to(device)

            for idxs in test_dataloader:
                smiles = pad_sequence([torch.tensor([BOS_ID]+smiles_list_test[idx]+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                labels = torch.tensor([labels_vec_list_test[idx] for idx in idxs],dtype=torch.float).to(device)

                estimated_labels = label_estimator(smiles)

                loss = criterion_label(estimated_labels, labels)

                total_loss += loss.item()*len(idxs)
                sample_num += len(idxs)

                if task_type == 'classification': 
                    total_tps += seek_TP(estimated_labels, labels).sum(0)
                    total_real_pos += (positive(labels).int()).sum(0)
                    total_pred_pos += (positive(estimated_labels).int()).sum(0)

                labels_test = torch.concat((labels_test, labels), dim=0)
                preds_test = torch.concat((preds_test, estimated_labels), dim=0)

            labels_test = labels_test[1:]
            preds_test = preds_test[1:]

            mean_loss = total_loss / sample_num
            list_test_loss.append(mean_loss)

            # compute metrics
            if task_type == 'classification':
                precision = total_tps/(total_pred_pos + 1e-9)
                recall    = total_tps/(total_real_pos + 1e-9)
                f1 = 2.0*precision*recall/(precision+recall+ 1e-9)
                eachODFold_test[_fold, epoch] = f1
                micro_mean_precision = total_tps.sum()/total_pred_pos.sum()
                micro_mean_recall    = total_tps.sum()/total_real_pos.sum()     
                metrics = 2*micro_mean_precision*micro_mean_recall/(micro_mean_precision+micro_mean_recall)
                metrics = metrics.item()
            else:
                metrics = math.sqrt(mean_loss)
                eachODFold_test[_fold, epoch, 0] = metrics

            list_test_metrics.append(metrics)

            avgOD_metrics_test[_fold, epoch] = metrics # record for tensorboard

            # plot on tensorboard
            writter.add_scalars('fold%s-microF1orRMSE/test' % _fold, {para_name: metrics}, epoch)
            
            macro_f1_fold = torch.sum(eachODFold_test[_fold, epoch, :]).item() / (lID)
            
            if task_type == 'classification':
                if macro_f1_fold > best_metrics_fold:
                    best_metrics_fold = macro_f1_fold
                    best_logits_pred_fold = preds_test.tolist() # shape=(batch, lID)
                    best_labels = labels_test.tolist()
                    best_epo_fold = epoch
            else:
                if macro_f1_fold < best_metrics_fold:
                    best_metrics_fold = macro_f1_fold
                    best_logits_pred_fold = preds_test.tolist() # shape=(batch, lID)
                    best_labels = labels_test.tolist()
            
            
            if _fold == fold_cv-1:
                macro_f1 = torch.sum(eachODFold_test[:, epoch, :]).item() / (fold_cv*lID)
                writter.add_scalars('AvgFold-macroF1orRMSE/test', {para_name: macro_f1}, epoch) 
                f1_eachod = torch.sum(eachODFold_test[:,epoch,:], dim=0) / fold_cv # shape=(lID, )
                for i in range(lID):
                    # od_f1 = torch.sum(eachODFold_test[:, epoch, i]).item() / fold_cv
                    writter.add_scalars('AvgFold-%s/test' % ID2label[i], {para_name: f1_eachod[i].item()}, epoch)

        best_metrics += best_metrics_fold


        # auc, r_square
        if task_type == 'classification':
            rocauc_macro += roc_auc_score(best_labels, best_logits_pred_fold, average='macro')
            pr_auc_macro += average_precision_score(best_labels, best_logits_pred_fold, average='macro')
            rocauc_micro += roc_auc_score(best_labels, best_logits_pred_fold, average='micro')
            pr_auc_micro += average_precision_score(best_labels, best_logits_pred_fold, average='micro')
            r_2 += avgOD_metrics_test[_fold, best_epo_fold].item()
        else:
            r_2 += r2_score(best_labels, best_logits_pred_fold)
        tdc_predictions_list.append({data_name: best_logits_pred_fold})

    
    result_dict = {'macroF1 or RMSE': best_metrics/fold_cv, 
    'ROCAUC_macro': rocauc_macro/fold_cv, 'ROCAUC_micro': rocauc_micro/fold_cv,
    'PRAUC macro': pr_auc_macro/fold_cv, 'PRAUC micro': pr_auc_micro/fold_cv, 
    'microF1 or r^2': r_2/fold_cv}

    
    writter.add_hparams(hpara,
                            result_dict, 
                            run_name=para_name
                            )

    tdc_result = tdc_group.evaluate_many(tdc_predictions_list)
    print(tdc_result)

    result_dict['tdc'] = tdc_result[data_name]

    return result_dict



## ODs

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import r2_score

def main_program_ODs(rate1, hpara):
    '''
    the learning rate for layers training in transfer learning is: LearningRate * rate1
    hpara = {
        'pretrain_path': 'modelsave0203',
        'DimEmbed': 256,
        'DimTfHidden': 512,
        'NumHead': 8,
        'NumLayers': 10,
        'MaskRate': 0.5,
        'LearningRate': 0.0001, 
        'Dropout': 0.1
    } 
    '''
    smiles_list, canonical_smiles_list,labels_vec_list,lID,label_ID,task_type,ID2label = dataTDC(data_name)
    
    
    torch.cuda.empty_cache()

    para_name = '%d_%d_%d_%d_%.2f_%.2f_%.2f' % (hpara['DimEmbed'], hpara['DimTfHidden'], 
    hpara['NumHead'], hpara['NumLayers'], hpara['MaskRate'], hpara['LearningRate'], hpara['Dropout'])

    list_train_loss = []
    list_train_metrics = [] # micro f1 or rmse
    list_test_loss = []
    list_test_metrics = [] # micro f1 or rmse

    if task_type == 'classfication':
        criterion_label = nn.BCELoss()
    else:
        criterion_label = nn.MSELoss()

    fold_cv = 5
    kf = KFold(n_splits=fold_cv, shuffle=True)


    dataset_idxs = list(range(len(smiles_list)))

    sum_train_micro_mean_f1 = 0 
    sum_test_micro_mean_f1 = 0
    num_lastpart = 0

    # record result for tensorboard
    # record F1 for classification and RMSE for regression
    avgOD_metrics = torch.zeros(fold_cv, NumEpoch).to(device) # microF1 or rmse
    eachODFold = torch.zeros(fold_cv, NumEpoch, lID).to(device) #macroF1 can be obtained by averaging lID axis

    avgOD_metrics_test = torch.zeros(fold_cv, NumEpoch).to(device) # microF1 or rmse
    eachODFold_test = torch.zeros(fold_cv, NumEpoch, lID).to(device) #macroF1 can be obtained by averaging lID axis


    # if task_type == 'classification':
    #     best_metrics = 0 # macro f1
    # else:
    #     best_metrics = 1e9 # rmse
    best_metrics = 0
    best_logits_pred = []

    # 改换单个smiles的root
    def change_smiles(smi):
        if smi.find('.') >= 0:
            return smiles_str2smiles(smi)

        mol = Chem.MolFromSmiles(smi)

        num_atoms = mol.GetNumAtoms()
        pos_list = list(range(num_atoms))

        pos = random.choice(pos_list)
        new_smi = Chem.MolToSmiles(mol, rootedAtAtom=pos)
        if len(new_smi) < LIMIT_SMILES_LENGTH:
            return smiles_str2smiles(new_smi)
        else:
            return smiles_str2smiles(smi)

    rocauc_macro = 0
    pr_auc_macro = 0
    rocauc_micro = 0
    pr_auc_micro = 0
    r_2 = 0

        
    for _fold, (train_idxs, test_idxs) in enumerate(kf.split(dataset_idxs)):
        #============================= train_dataloader ===========================
        train_dataset_idxs = Subset(dataset_idxs, train_idxs)
        train_dataloader = DataLoader(train_dataset_idxs, batch_size, shuffle=True)

        #============================== test_dataloader =====================================
        test_dataset_idxs  = Subset(dataset_idxs, test_idxs)
        test_dataloader = DataLoader(test_dataset_idxs, batch_size, shuffle=True)  

        #====================== モデルの取得、optimizer ==============
        label_estimator = Model(lID, task_type).to(device)


        # 事前学習済みの smiles_encoder のパラメタを読み込む
        print('fold = ', _fold)

        if hpara['pretrain_path'] == 'nopre':
            optimizer_label_estimator = optim.Adam(label_estimator.parameters(), hpara['LearningRate'])
        else:
            pf_symbol_en = hpara['pretrain_path']+'/'+'OdorCode-40 symbol_encoder D'+str(hpara['DimEmbed'])+'.Hidden'+str(hpara['DimTfHidden'])+'.Head'+ str(hpara['NumHead'])+'.L'+str(hpara['NumLayers'])+'.R'+str(hpara['MaskRate'])+'.S'+str(TotalSize)+'-epoch.'+str(ModelEpoch)
            pf_smiles_en = hpara['pretrain_path']+'/'+'OdorCode-40 smiles_encoder D'+str(hpara['DimEmbed'])+'.Hidden'+str(hpara['DimTfHidden'])+'.Head'+ str(hpara['NumHead'])+'.L'+str(hpara['NumLayers'])+'.R'+str(hpara['MaskRate'])+'.S'+str(TotalSize)+'-epoch.'+str(ModelEpoch)
            print('reading '+pf_symbol_en)
            label_estimator.symbol_encoder.load_state_dict(torch.load(pf_symbol_en))
            print('reading '+pf_smiles_en)
            label_estimator.smiles_encoder.load_state_dict(torch.load(pf_smiles_en))

            for param in label_estimator.symbol_encoder.parameters():
                param.requires_grad = False

            for param in label_estimator.smiles_encoder.parameters():
                param.requires_grad = False
            for param in label_estimator.smiles_encoder.transformer_encoder.layers[-1].parameters():
                param.requires_grad = True
            
            optimizer_label_estimator = optim.Adam([ {'params': label_estimator.smiles_encoder.transformer_encoder.layers[-1].parameters(), 'lr': hpara['LearningRate']*rate1},
                                                        {'params': label_estimator.fnn2.parameters(), 'lr': hpara['LearningRate']},
                                                        {'params': label_estimator.fnn.parameters(), 'lr': hpara['LearningRate']}])
                                                      {'params': label_estimator.classifier.parameters(), 'lr': hpara['LearningRate']}])

        if task_type == 'classification':
            best_metrics_fold = 0 # macro f1
        else:
            best_metrics_fold = 1e9 # rmse
        best_logits_pred_fold = []
        best_epo_fold = None

        # rocauc_macro = 0
        # pr_auc_macro = 0
        # rocauc_micro = 0
        # pr_auc_micro = 0
        # r_2 = 0

        for epoch in range(NumEpoch):
            #---------------- train step -----------------
            sample_num = 0
            total_loss = 0

            if task_type == 'classification': 
                total_tps = torch.zeros(lID).to(device)
                total_real_pos = torch.zeros(lID).to(device)
                total_pred_pos = torch.zeros(lID).to(device)

            label_estimator.train()

            for idxs in train_dataloader:
                # change smiles root
                smiles = pad_sequence([torch.tensor([BOS_ID]+change_smiles(canonical_smiles_list[idx])+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                # smiles = pad_sequence([torch.tensor([BOS_ID]+smiles_list[idx]+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                labels = torch.tensor([labels_vec_list[idx] for idx in idxs],dtype=torch.float).to(device)

                optimizer_label_estimator.zero_grad()

                estimated_labels = label_estimator(smiles)

                loss = criterion_label(estimated_labels, labels)
                
                loss.backward()     

                optimizer_label_estimator.step()  

                total_loss += loss.item()*len(idxs)
                sample_num += len(idxs)

                if task_type == 'classification': 
                    total_tps += seek_TP(estimated_labels, labels).sum(0)
                    total_real_pos += (positive(labels).int()).sum(0)
                    total_pred_pos += (positive(estimated_labels).int()).sum(0)

            mean_loss = total_loss / sample_num
            list_train_loss.append(mean_loss)

            # metrics计算
            if task_type == 'classification':
                precision = total_tps/(total_pred_pos + 1e-9)
                recall    = total_tps/(total_real_pos + 1e-9)
                f1 = 2.0*precision*recall/(precision+recall+1e-9) # (lID, )
                eachODFold[_fold, epoch] = f1
                micro_mean_precision = total_tps.sum()/total_pred_pos.sum()
                micro_mean_recall    = total_tps.sum()/total_real_pos.sum()      
                metrics = 2*micro_mean_precision*micro_mean_recall/(micro_mean_precision+micro_mean_recall)
                metrics = metrics.item()
            else:
                metrics = math.sqrt(mean_loss)
                eachODFold[_fold, epoch, 0] = metrics 

            list_train_metrics.append(metrics)

            avgOD_metrics[_fold, epoch] = metrics # record for tensorboard
            
            # plot on tensorboard
            writter.add_scalars('fold%s-microF1orRMSE/train' % _fold, {para_name: metrics}, epoch)
            if _fold == fold_cv-1:
                macro_f1 = torch.sum(eachODFold[:, epoch, :]).item() / (fold_cv*lID)
                writter.add_scalars('AvgFold-macroF1orRMSE/train', {para_name: macro_f1}, epoch) 
                for i in range(lID):
                    od_f1 = torch.sum(eachODFold[:, epoch, i]).item() / fold_cv
                    writter.add_scalars('AvgFold-%s/train' % ID2label[i], {para_name:od_f1}, epoch)


            #--------------- test step ------------------
            sample_num = 0
            total_loss = 0

            if task_type == 'classification':
                total_tps = torch.zeros(lID).to(device)
                total_real_pos = torch.zeros(lID).to(device)
                total_pred_pos = torch.zeros(lID).to(device)

            label_estimator.eval()

            labels_test = torch.zeros([1, lID]).to(device)
            preds_test = torch.zeros([1, lID]).to(device)

            for idxs in test_dataloader:
                smiles = pad_sequence([torch.tensor([BOS_ID]+smiles_list[idx]+[EOS_ID]) for idx in idxs], batch_first=True).to(device)
                labels = torch.tensor([labels_vec_list[idx] for idx in idxs],dtype=torch.float).to(device)

                estimated_labels = label_estimator(smiles)

                loss = criterion_label(estimated_labels, labels)

                total_loss += loss.item()*len(idxs)
                sample_num += len(idxs)

                if task_type == 'classification': 
                    total_tps += seek_TP(estimated_labels, labels).sum(0)
                    total_real_pos += (positive(labels).int()).sum(0)
                    total_pred_pos += (positive(estimated_labels).int()).sum(0)

                labels_test = torch.concat((labels_test, labels), dim=0)
                preds_test = torch.concat((preds_test, estimated_labels), dim=0)

            labels_test = labels_test[1:]
            preds_test = preds_test[1:]

            mean_loss = total_loss / sample_num
            list_test_loss.append(mean_loss)

            # metrics
            if task_type == 'classification':
                precision = total_tps/(total_pred_pos + 1e-9)
                recall    = total_tps/(total_real_pos + 1e-9)
                f1 = 2.0*precision*recall/(precision+recall+ 1e-9)
                eachODFold_test[_fold, epoch] = f1
                micro_mean_precision = total_tps.sum()/total_pred_pos.sum()
                micro_mean_recall    = total_tps.sum()/total_real_pos.sum()     
                metrics = 2*micro_mean_precision*micro_mean_recall/(micro_mean_precision+micro_mean_recall)
                metrics = metrics.item()
            else:
                metrics = math.sqrt(mean_loss)
                eachODFold_test[_fold, epoch, 0] = metrics

            list_test_metrics.append(metrics)

            avgOD_metrics_test[_fold, epoch] = metrics # record for tensorboard

            # plot on tensorboard
            writter.add_scalars('fold%s-microF1orRMSE/test' % _fold, {para_name: metrics}, epoch)
            
            macro_f1_fold = torch.sum(eachODFold_test[_fold, epoch, :]).item() / (lID)
            
            if task_type == 'classification':
                if macro_f1_fold > best_metrics_fold:
                    best_metrics_fold = macro_f1_fold
                    best_logits_pred_fold = preds_test.tolist() # shape=(batch, lID)
                    best_labels = labels_test.tolist()
                    best_epo_fold = epoch
            else:
                if macro_f1_fold < best_metrics_fold:
                    best_metrics_fold = macro_f1_fold
                    best_logits_pred_fold = preds_test.tolist() # shape=(batch, lID)
                    best_labels = labels_test.tolist()
            
            
            if _fold == fold_cv-1:
                macro_f1 = torch.sum(eachODFold_test[:, epoch, :]).item() / (fold_cv*lID)
                writter.add_scalars('AvgFold-macroF1orRMSE/test', {para_name: macro_f1}, epoch) 
                f1_eachod = torch.sum(eachODFold_test[:,epoch,:], dim=0) / fold_cv # shape=(lID, )
                for i in range(lID):
                    # od_f1 = torch.sum(eachODFold_test[:, epoch, i]).item() / fold_cv
                    writter.add_scalars('AvgFold-%s/test' % ID2label[i], {para_name: f1_eachod[i].item()}, epoch)

                

        best_metrics += best_metrics_fold
            
        if fold_1:
            break


        # auc, r_square
        if task_type == 'classification':
            rocauc_macro += roc_auc_score(best_labels, best_logits_pred_fold, average='macro')
            pr_auc_macro += average_precision_score(best_labels, best_logits_pred_fold, average='macro')
            rocauc_micro += roc_auc_score(best_labels, best_logits_pred_fold, average='micro')
            pr_auc_micro += average_precision_score(best_labels, best_logits_pred_fold, average='micro')
            r_2 += avgOD_metrics_test[_fold, best_epo_fold].item()
        else:
            r_2 += r2_score(best_labels, best_logits_pred_fold)

    
    result_dict = {'macroF1 or RMSE': best_metrics/fold_cv, 
    'ROCAUC_macro': rocauc_macro/fold_cv, 'ROCAUC_micro': rocauc_micro/fold_cv,
    'PRAUC macro': pr_auc_macro/fold_cv, 'PRAUC micro': pr_auc_micro/fold_cv, 
    'microF1 or r^2': r_2/fold_cv}

    
    writter.add_hparams(hpara,
                            result_dict, 
                            run_name=para_name
                            )

    return result_dict


# experiment

## modelsave0203

In [ ]:
torch.cuda.empty_cache()

hpara = {
        'pretrain_path': 'modelsave0203',
        'DimEmbed': 256,
        'DimTfHidden': 512,
        'NumHead': 8,
        'NumLayers': 10,
        'MaskRate': 0.5,
        'LearningRate': 0.0001, #learning rate for transfer learning instead of pretraining
        'Dropout': 0.1
    } 

# hyperparameters for pretraining (used for loading pretrained model)
DimEmbed = hpara['DimEmbed']     # dimension of embedding
DimTfHidden = hpara['DimTfHidden']  # dimension of FNN in TransformerEncoder 
NumHead = hpara['NumHead']        #the number of heads of multi-head in  TransformerEncoder
NumLayers = hpara['NumLayers']      #the number of layers of TransformerEncoder
NormFirst = True   # Vision Transformer 
Activation = 'gelu' 

MaskRate = hpara['MaskRate']
TotalSize = 100000
ModelEpoch = 600

# hyperparameters for transfer learning
LearningRate = hpara['LearningRate']
NumEpoch = 50
Dropout = hpara['Dropout']
batch_size = 32
InitRange = 0.1

# TDC prediction
for data_name in data_name_list:
    # writter
    record_dict = 'tensorboard_logs/torch/240329/%s/' % data_name
    writter = SummaryWriter(record_dict)

    result_dict = main_program(1.0, hpara, fold_cv=5, data_name=data_name)

    writter.close()

    print(result_dict)

    #csv
    file_txt = 'resultTXT/240329/'
    file_name = 'tdcG-enc.csv'
    logResult(file_txt, 
                file_name, 
                hpara['pretrain_path'],
                data_name,
                result_dict
                )

# ODs prediction
# # writter
# record_dict = 'tensorboard_logs/torch/ODs/'
# writter = SummaryWriter(record_dict)

# main_program_ODs(1.0, hpara)

# writter.close()

## non-pretrain

In [ ]:
DimEmbed_list = [128,256]
NumHead_list = [8]
NumLayers_list = [8, 10]
Dropout_list = [0.1]

for DimEmbed in DimEmbed_list:
    DimTfHidden = DimEmbed
    for NumHead in NumHead_list:
        for NumLayers in NumLayers_list:
            for Dropout in Dropout_list:
                hpara = {
                    'pretrain_path': 'nopre',
                    'DimEmbed': DimEmbed,
                    'DimTfHidden': DimTfHidden,
                    'NumHead': NumHead,
                    'NumLayers': NumLayers,
                    'MaskRate': 0.5,
                    'LearningRate': 0.0003,
                    'Dropout': Dropout
                } 

                NormFirst = True   # Vision Transformer 
                Activation = 'gelu' 
                
                LearningRate = hpara['LearningRate']
                NumEpoch = 150
                batch_size = 64
                InitRange = 0.1

                # TDC prediction
                for data_name in data_name_list:
                    # writter
                    record_dict = 'tensorboard_logs/torch/240409-nopre/%s/' % data_name
                    writter = SummaryWriter(record_dict)

                    result_dict = main_program(1.0, hpara, fold_cv=1, data_name=data_name)

                    writter.close()

                    print(result_dict)

                    #csv
                    model_name = hpara['pretrain_path']+'%d_%d_%d_%d' % (hpara['DimEmbed'], hpara['DimTfHidden'], hpara['NumHead'], hpara['NumLayers'])

                    file_txt = 'resultTXT/240409/'
                    file_name = 'tdc_nonpretrain.csv'
                    logResult(file_txt, 
                                file_name, 
                                model_name,
                                data_name,
                                result_dict
                                ) 

                # ODs prediction
                # # writter
                # record_dict = 'tensorboard_logs/torch/ODs-nonpretrain/'
                # writter = SummaryWriter(record_dict)

                # main_program_ODs(1.0, hpara)

                # writter.close()      